# Important: Run this code cell each time you start a new session!

In [1]:
# Do not edit this cell
def test_homework(test_name, actual, expected):
  if actual == expected:
    print(f"Test passed: {test_name}.")
    return 1
  else:
    print(f"Test failed: {test_name}. Expected {expected}, got {actual}")
    return 0

def compare_hw_scores(score, max_score):
  if score == max_score:
    print("All test cases passed!")
  print(f"Mark: {score} / {max_score}")

In [2]:
import os, datetime, json, locale, pathlib, urllib, requests, werkzeug, nbformat, google, yaml, warnings
def colab2pdf():
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
    NAME = pathlib.Path(werkzeug.utils.secure_filename(urllib.parse.unquote(requests.get(f"http://{os.environ['COLAB_JUPYTER_IP']}:{os.environ['KMP_TARGET_PORT']}/api/sessions").json()[0]["name"])))
    TEMP = pathlib.Path("/content/pdfs") / f"{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_{NAME.stem}"; TEMP.mkdir(parents=True, exist_ok=True)
    NB = [cell for cell in nbformat.reads(json.dumps(google.colab._message.blocking_request("get_ipynb", timeout_sec=30)["ipynb"]), as_version=4).cells if "--Colab2PDF" not in cell.source]
    warnings.filterwarnings('ignore', category=nbformat.validator.MissingIDFieldWarning)
    with (TEMP / f"{NAME.stem}.ipynb").open("w", encoding="utf-8") as nb_copy: nbformat.write(nbformat.v4.new_notebook(cells=NB or [nbformat.v4.new_code_cell("#")]), nb_copy)
    if not pathlib.Path("/usr/local/bin/quarto").exists():
        !wget -q "https://quarto.org/download/latest/quarto-linux-amd64.deb" -P {TEMP} && dpkg -i {TEMP}/quarto-linux-amd64.deb > /dev/null && quarto install tinytex --update-path --quiet
    with (TEMP / "config.yml").open("w", encoding="utf-8") as file: yaml.dump({'include-in-header': [{"text": r"\usepackage{fvextra}\DefineVerbatimEnvironment{Highlighting}{Verbatim}{breaksymbolleft={},showspaces=false,showtabs=false,breaklines,breakanywhere,commandchars=\\\{\}}"}],'include-before-body': [{"text": r"\DefineVerbatimEnvironment{verbatim}{Verbatim}{breaksymbolleft={},showspaces=false,showtabs=false,breaklines}"}]}, file)
    !quarto render {TEMP}/{NAME.stem}.ipynb --metadata-file={TEMP}/config.yml --to pdf -M latex-auto-install -M margin-top=1in -M margin-bottom=1in -M margin-left=1in -M margin-right=1in --quiet
    google.colab.files.download(str(TEMP / f"{NAME.stem}.pdf"))

# Instructions

Each exercise generally consists of four parts.
1.   A description of the function you need to implement
2.   Any starter code to help you write your code
3.   A separate code block where you can test out various parts of your code
4.   An autograder that will evaluate your code against some test cases we have selected

The cell after all of the exercises will confirm that you have successfully completed the homework. This cell will not run unless you run the autograder for each individual exercise first.

# Exercise 1: Oversubscribed Physicians


Consider a dictionary that stores the list of patients that meets with each physician at a clinic; the physician names are the keys and the list of those physician's patients are the values. For example, a dictionary of this form is:

```
patient_data = {"Dr. House": ["Alice", "Adam", "Aaron"],
                "Dr. McCoy": ["Bob", "Beatrice"],
                "Dr. Livesey": ["Chloe", "Cameron", "Constantine"]}
```

Complete the following function according to its docstring.

In [3]:
def oversubscribed_physicians(patient_data, max_patients):
    """(dict, int) -> list of str

    Return a list of the physicians who have more than
    max_patients patients, as recorded in patient_data,
    with the names sorted alphabetically. In patient_data,
    each key is a physician and each value is a list of that
    physician's patients.

    >>> patient_data = {"Dr. House": ["Alice", "Adam", "Aaron"],
                        "Dr. McCoy": ["Bob", "Beatrice"],
                        "Dr. Livesey": ["Chloe", "Cameron", "Constantine"]}
    >>> oversubscribed_physicians(patient_data, 2)
    ["Dr. House", "Dr. Livesey"]
    >>> oversubscribed_physicians(patient_data, 3)
    []
    """
    result = []
    for physician in patient_data:
        physician_patient_list = patient_data[physician]
        if len(physician_patient_list) > max_patients:
            result.append(physician)
    return sorted(result)

In [4]:
patient_data = {"Dr. House": ["Alice", "Adam", "Aaron"],
                "Dr. McCoy": ["Bob", "Beatrice"],
                "Dr. Livesey": ["Chloe", "Cameron", "Constantine"]}
print(oversubscribed_physicians(patient_data, 2))
print(oversubscribed_physicians(patient_data, 3))

['Dr. House', 'Dr. Livesey']
[]


#### Run the hidden code cell to evaluate Exercise 1

In [5]:
# Do not edit this cell
def check_exercise1():
  ex_score, max_ex_score = 0, 0

  ex_score += test_homework("All have less than or the same as max", oversubscribed_physicians({"Dr. House": ["Alice", "Adam", "Aaron"], "Dr. McCoy": ["Bob", "Beatrice"], "Dr. Livesey": ["Chloe", "Cameron", "Constantine", "Cleopatra"]} , 4), [])
  max_ex_score += 1
  ex_score += test_homework("All have more than max", oversubscribed_physicians({"D": ["a", "b", "c"], "B": ["d", "e"], "C": ["f", "g"], "A":["h", "i", "j", "k"]}, 1), ["A", "B", "C", "D"])
  max_ex_score += 1
  ex_score += test_homework("Mix of cases", oversubscribed_physicians( {"Dr. House": ["Alice", "Adam", "Aaron"], "Dr. McCoy": ["Bob", "Beatrice"], "Dr. Livesey": ["Chloe", "Cameron", "Constantine", "Cleopatra"]} , 2), ["Dr. House", "Dr. Livesey"])
  max_ex_score += 1

  compare_hw_scores(ex_score, max_ex_score)
  return ex_score, max_ex_score

_ = check_exercise1()

Test passed: All have less than or the same as max.
Test passed: All have more than max.
Test passed: Mix of cases.
All test cases passed!
Mark: 3 / 3


# Exercise 2: Updating Patient Priority

Consider a dictionary that keeps track of every patient's status in order to determine who needs the most attention. Each patient is assigned a numerical priority rating. Higher ratings indicate more severe cases, and ratings above 10 require immediate attention. In the following example:

```
waiting_patients = {"Alice": 4, "Bob": 11, "Chloe": 5}
```

the patients in descending order of priority are Bob, Chloe, and Alice; however, Bob requires immediate attention since his rating is above 10.

Complete the following function according to its docstring.

**Important:** This function should simultaneously modify the original dictionary `waiting_patients` and return a string.

In [6]:
def update_priority(waiting_patients, patient, change):
    """ (dict{str: int}, str, int) -> str

    Change the priority of patient in the dictionary waiting_patients
    by adding change, which could be positive or negative.

    If this causes the priority of this patient to be greater than 10,
    return the string "alert", otherwise return the string "normal". If
    patient is not in waiting_patients, add in the patient with the priority
    value of change.

    >>> priorities_dict = {"P1":4, "P2":5}
    >>> ret = update_priority(priorities_dict, "P1", 2)
    >>> priorities_dict
    {"P1":6, "P2":5}
    >>> ret
    "normal"

    >>> ret = update_priority(priorities_dict, "P2", 7)
    >>> priorities_dict
    {"P1":6, "P2":12}
    >>> ret
    "alert"
    """
    # Check if the person isn't in the list already
    # Or else update their priority
    if patient not in waiting_patients:
        waiting_patients[patient] = change
    else:
        waiting_patients[patient] += change

    # Check priority
    if waiting_patients[patient] > 10:
        return "alert"
    else:
        return "normal"

In [7]:
priorities_dict = {"P1":4, "P2":5}
ret = update_priority(priorities_dict, "P1", 2)
print(priorities_dict)
print(ret)

ret = update_priority(priorities_dict, "P2", 7)
print(priorities_dict)
print(ret)

{'P1': 6, 'P2': 5}
normal
{'P1': 6, 'P2': 12}
alert


#### Run the hidden code cell to evaluate Exercise 2

In [8]:
# Do not edit this cell
def check_exercise2():
  ex_score, max_ex_score = 0, 0

  priorities_dict = {"P1":4, "P2":5}
  ret = update_priority(priorities_dict, "P1", 2)
  result = priorities_dict["P1"] == 6 and priorities_dict["P2"] == 5 and ret == "normal"
  ex_score += test_homework("Increasing existing patient to less than 10", result, True)
  max_ex_score += 1

  priorities_dict = {"P1":4, "P2":5}
  ret = update_priority(priorities_dict, "P2", 7)
  result = priorities_dict["P1"] == 4 and priorities_dict["P2"] == 12 and ret == "alert"
  ex_score += test_homework("Increasing existing patient to > 10", result, True)
  max_ex_score += 1

  priorities_dict = {"P1":4, "P2":5}
  ret = update_priority(priorities_dict, "P3", 2)
  result = priorities_dict == {"P1":4, "P2":5, "P3":2} and ret == "normal"
  ex_score += test_homework("Adding patient with less than 10", result, True)
  max_ex_score += 1

  priorities_dict = {"P1":4, "P2":5}
  ret = update_priority(priorities_dict, "P3", 12)
  result = priorities_dict == {"P1":4, "P2":5, "P3":12} and ret == "alert"
  ex_score += test_homework("Adding patient with > 10", result, True)
  max_ex_score += 1

  priorities_dict = {"P1":4, "P2":5}
  ret = update_priority(priorities_dict, "P1", 6)
  result = priorities_dict["P1"] == 10 and priorities_dict["P2"] == 5 and ret == "normal"
  ex_score += test_homework("Increasing existing patient to exactly 10", result, True)
  max_ex_score += 1

  compare_hw_scores(ex_score, max_ex_score)
  return ex_score, max_ex_score

_ = check_exercise2()

Test passed: Increasing existing patient to less than 10.
Test passed: Increasing existing patient to > 10.
Test passed: Adding patient with less than 10.
Test passed: Adding patient with > 10.
Test passed: Increasing existing patient to exactly 10.
All test cases passed!
Mark: 5 / 5


# Exercise 3: Check Symptoms


Consider a dictionary that stores the symptoms that everyone experienced during a clinical trial. For example, the dictionary might look as follows:

```
symptom_dict = {"Alice Z.": ["fever"],
                "Bob Y.": ["nausea", "lightheadedness"],
                "Carl X.": ["fever", "nausea"],
                "Diana W.": [],
                "Eve V.": ["headache", "nausea"],
                "Frank U.": ["fever", "nausea"],
                "Greg T.": ["gangrene"]}
```

Write a function that returns `True` iff none of the patients exhibited any symptoms; otherwise, return `False`.

In [9]:
def no_symptoms_found(symptom_dict):
    """ (dict) -> bool

    symptom_dict is a dictionary where the keys are patient names
    and the values are a list of the symptoms that they exhibit.

    Return True iff none of the patients have any symptoms;
    otherwise, return False.

    >>> symptom_dict1 = {"Alice Z.": ["fever"],
                         "Bob Y.": ["nausea", "lightheadedness"],
                         "Carl X.": ["fever", "nausea"],
                         "Diana W.": [],
                         "Eve V.": ["headache", "nausea"],
                         "Frank U.": ["fever", "nausea"],
                         "Greg T.": ["gangrene"]}
    >>> no_symptoms_found(symptom_dict1)
    False
    >>> symptom_dict2 = {"Alice Z.": [], "Bob Y.": [], "Carl X.": [],
                         "Diana W.": [], "Eve V.": [],  "Frank U.": [],
                         "Greg T.": []}
    >>> no_symptoms_found(symptom_dict2)
    True
    """
    for patient in symptom_dict:
        if len(symptom_dict[patient]) > 0:
            return False
    return True

In [10]:
symptom_dict1 = {"Alice Z.": ["fever"],
                "Bob Y.": ["nausea", "lightheadedness"],
                "Carl X.": ["fever", "nausea"],
                "Diana W.": [],
                "Eve V.": ["headache", "nausea"],
                "Frank U.": ["fever", "nausea"],
                "Greg T.": ["gangrene"]}
print(no_symptoms_found(symptom_dict1))

symptom_dict2 = {"Alice Z.": [], "Bob Y.": [], "Carl X.": [],
                 "Diana W.": [], "Eve V.": [],  "Frank U.": [],
                 "Greg T.": []}
print(no_symptoms_found(symptom_dict2))

False
True


#### Run the hidden code cell to evaluate Exercise 3

In [11]:
# Do not edit this cell
def check_exercise3():
  ex_score, max_ex_score = 0, 0

  ex_score += test_homework("No patients", no_symptoms_found({}), True)
  max_ex_score += 1
  ex_score += test_homework("One patient, no symptoms", no_symptoms_found({"a": []}), True)
  max_ex_score += 1
  ex_score += test_homework("Two patients no symptoms", no_symptoms_found({"a": [], "b": []}), True)
  max_ex_score += 1
  ex_score += test_homework("One patient, symptoms", no_symptoms_found({"a": ["1"]}), False)
  max_ex_score += 1
  ex_score += test_homework("Mostly no symptoms", no_symptoms_found({"a": [], "b": [], "c": [], "d": [], "e":["1", "2", "3"]}), False)
  max_ex_score += 1

  compare_hw_scores(ex_score, max_ex_score)
  return ex_score, max_ex_score

_ = check_exercise3()

Test passed: No patients.
Test passed: One patient, no symptoms.
Test passed: Two patients no symptoms.
Test passed: One patient, symptoms.
Test passed: Mostly no symptoms.
All test cases passed!
Mark: 5 / 5


# Exercise 4: Finding Dangerous Symptoms

Consider the same dictionary of symptoms from Exercise 3 and a list of dangerous symptoms called `dangerous_symptoms`. This time, create a function that returns `True` iff any of the patients exhibit any of the symptoms in that list; otherwise, return `False`.

In [12]:
def dangerous_symptoms_found(symptom_dict, dangerous_symptoms):
    """
    (dict, list of str) -> bool

    symptom_dict is a dictionary where the keys are patient names
    and the values are a list of the symptoms that they exhibit.

    Return True iff none of the patients have any of the harmful
    symptoms listed in dangerous_symptoms; otherwise, return False.

    >>> symptom_dict = {"Alice Z.": ["fever"],
                        "Bob Y.": ["nausea", "lightheadedness"],
                        "Carl X.": ["fever", "nausea"],
                        "Diana W.": [],
                        "Eve V.": ["headache", "nausea"],
                        "Frank U.": ["fever", "nausea"],
                        "Greg T.": ["gangrene"]}
    >>> dangerous_symptoms = ["gangrene", "cardiac arrest"]
    >>> dangerous_symptoms_found(symptom_dict, dangerous_symptoms)
    True
    >>> symptom_dict = {"Alice Z.": ["fever"],
                        "Bob Y.": ["nausea", "lightheadedness"],
                        "Diana W.": []}
    >>> dangerous_symptoms_found(symptom_dict, dangerous_symptoms)
    False
    """
    for patient in symptom_dict:
        patient_symptoms = symptom_dict[patient]
        for symptom in patient_symptoms:
            if symptom in dangerous_symptoms:
                return True
    return False

In [13]:
symptom_dict = {"Alice Z.": ["fever"],
                "Bob Y.": ["nausea", "lightheadedness"],
                "Carl X.": ["fever", "nausea"],
                "Diana W.": [],
                "Eve V.": ["headache", "nausea"],
                "Frank U.": ["fever", "nausea"],
                "Greg T.": ["gangrene"]}
dangerous_symptoms = ["gangrene", "cardiac arrest"]
print(dangerous_symptoms_found(symptom_dict, dangerous_symptoms))

symptom_dict = {"Alice Z.": ["fever"],
                "Bob Y.": ["nausea", "lightheadedness"],
                "Diana W.": []}
print(dangerous_symptoms_found(symptom_dict, dangerous_symptoms))

True
False


#### Run the hidden code cell to evaluate Exercise 4

In [14]:
# Do not edit this cell
def check_exercise4():
  ex_score, max_ex_score = 0, 0

  symptom_dict = {"Alice Z.": ["fever"], "Bob Y.": ["nausea", "lightheadedness"], "Carl X.": ["fever", "nausea"], "Diana W.": [], "Eve V.": ["headache", "nausea"], "Frank U.": ["fever", "nausea"], "Greg T.": ["gangrene"]}
  dangerous_symptoms = ["gangrene", "cardiac arrest"]
  copy_ds = dangerous_symptoms[:]
  result = dangerous_symptoms_found(symptom_dict, dangerous_symptoms)
  ex_score += test_homework("From problem description 1", (copy_ds == dangerous_symptoms) and result, True)
  max_ex_score += 1

  symptom_dict = {"a": [], "b": ["1"], "c": ["2", "3"]}
  dangerous = ["1"]
  result = dangerous_symptoms_found(symptom_dict,dangerous)
  ex_score += test_homework("One dangerous symptom", result, True)
  max_ex_score += 1

  symptom_dict = {"a": [], "b": ["1"], "c": ["2", "3"]}
  dangerous = ["4"]
  result = dangerous_symptoms_found(symptom_dict, dangerous)
  ex_score += test_homework("No dangerous symptoms", result, False)
  max_ex_score += 1

  symptom_dict = {"a": [], "b": ["1"], "c": ["2", "3"]}
  dangerous = ["3"]
  result = dangerous_symptoms_found(symptom_dict, dangerous)
  ex_score += test_homework("Dangerous symptom not first in list", result, True)
  max_ex_score += 1

  symptom_dict = {"a": [], "b": ["1"], "c": ["2", "3"]}
  dangerous = []
  result = dangerous_symptoms_found(symptom_dict, dangerous)
  ex_score += test_homework("No dangerous symptoms exist at all", result, False)
  max_ex_score += 1

  symptom_dict = {"a": [], "b": ["1"], "c": ["2", "3"]}
  dangerous = ['4', '5', '6', '7', '3']
  result = dangerous_symptoms_found(symptom_dict, dangerous)
  ex_score += test_homework("First dangerous symptom not found, but later one is found", result, True)
  max_ex_score += 1

  compare_hw_scores(ex_score, max_ex_score)
  return ex_score, max_ex_score

_ = check_exercise4()

Test passed: From problem description 1.
Test passed: One dangerous symptom.
Test passed: No dangerous symptoms.
Test passed: Dangerous symptom not first in list.
Test passed: No dangerous symptoms exist at all.
Test passed: First dangerous symptom not found, but later one is found.
All test cases passed!
Mark: 6 / 6


# Exercise 5: Managing Medications

Managing all the medications in a hospital is quite a challenge. Patients may require a number of medications, and hospitals must make sure they have enough of each medication for all the patients who require it. In this question, we have a dictionary connecting patients to the lists of their medications. Your task is to write a function that produces a dictionary where each key is a medication and each value is the list of patients who require that medication.

Complete the function below according to its docstring. To help you solve this problem, you may want to refer to the following tutorial video on how to invert a dictionary: https://play.library.utoronto.ca/watch/ZNuabdtIfubt

In [15]:
def reorder_by_medication(patient_to_medications):
    """ (dict of {str: list of str}) -> dict of {str: list of str}

    Return a dictionary that maps the names of each medication to all of the
    patients that require it.

    >>> reorder_by_medication({'Patient1': ["Lasix", "Ativan"], 'Patient2': ["Ativan"]})
    {'Lasix':['Patient1'], 'Ativan':['Patient1', 'Patient2']}
    """
    medication_to_patients = {}
    for patient in patient_to_medications:
        patients_medications = patient_to_medications[patient]
        for medication in patients_medications:
            # Create an empty list if the medication hasn't been seen
            if medication not in medication_to_patients:
                medication_to_patients[medication] = []
            # Add the patient to the list
            medication_to_patients[medication].append(patient)
    return medication_to_patients

In [16]:
reorder_by_medication({'Patient1': ["Lasix", "Ativan"], 'Patient2': ["Ativan"]})

{'Lasix': ['Patient1'], 'Ativan': ['Patient1', 'Patient2']}

#### Run the hidden code cell to evaluate Exercise 5


In [17]:
# Do not edit this cell
def check_exercise5():
	ex_score, max_ex_score = 0, 0

	ex_score += test_homework("One patient, one med", reorder_by_medication({'Patient1': ["Lasix"]}), {'Lasix': ['Patient1']})
	max_ex_score += 1

	ex_score += test_homework("One patient, multiple meds", reorder_by_medication({'Patient1':["Lasix", "Ativan"]}), {'Lasix':['Patient1'], 'Ativan':['Patient1']})
	max_ex_score += 1

	output = reorder_by_medication({'Patient1':["Ativan"], 'Patient2':["Ativan"]})
	for val in output.values():
		val.sort()
	ex_score += test_homework("One med, multiple patients", output, {'Ativan' : ['Patient1', 'Patient2']})
	max_ex_score += 1

	output = reorder_by_medication({'Patient1':["Lasix", "Ativan"], 'Patient2':["Ativan"], 'Patient3': [], 'Patient4': ['Lasix']})
	for val in output.values():
		val.sort()
	ex_score += test_homework("Multiple patients, multiple meds",output, {'Lasix':['Patient1','Patient4'] , 'Ativan':['Patient1', 'Patient2']})
	max_ex_score += 1

	compare_hw_scores(ex_score, max_ex_score)
	return ex_score, max_ex_score

_ = check_exercise5()

Test passed: One patient, one med.
Test passed: One patient, multiple meds.
Test passed: One med, multiple patients.
Test passed: Multiple patients, multiple meds.
All test cases passed!
Mark: 4 / 4


# Check Homework Completion

Run this cell to make sure you have completed all of the exercises.

In [18]:
# Do not edit this cell
hw_score, max_hw_score = 0, 0

try:
  ex1_score, max_ex1_score = check_exercise1()
  hw_score += ex1_score
  max_hw_score += max_ex1_score

  ex2_score, max_ex2_score = check_exercise2()
  hw_score += ex2_score
  max_hw_score += max_ex2_score

  ex3_score, max_ex3_score = check_exercise3()
  hw_score += ex3_score
  max_hw_score += max_ex3_score

  ex4_score, max_ex4_score = check_exercise4()
  hw_score += ex4_score
  max_hw_score += max_ex4_score

  ex5_score, max_ex5_score = check_exercise5()
  hw_score += ex5_score
  max_hw_score += max_ex5_score

except NameError:
  raise Exception("Autograder failed to run. You have either not completed all of the exercises or did not run the entire notebook")

compare_hw_scores(hw_score, max_hw_score)

Test passed: All have less than or the same as max.
Test passed: All have more than max.
Test passed: Mix of cases.
All test cases passed!
Mark: 3 / 3
Test passed: Increasing existing patient to less than 10.
Test passed: Increasing existing patient to > 10.
Test passed: Adding patient with less than 10.
Test passed: Adding patient with > 10.
Test passed: Increasing existing patient to exactly 10.
All test cases passed!
Mark: 5 / 5
Test passed: No patients.
Test passed: One patient, no symptoms.
Test passed: Two patients no symptoms.
Test passed: One patient, symptoms.
Test passed: Mostly no symptoms.
All test cases passed!
Mark: 5 / 5
Test passed: From problem description 1.
Test passed: One dangerous symptom.
Test passed: No dangerous symptoms.
Test passed: Dangerous symptom not first in list.
Test passed: No dangerous symptoms exist at all.
Test passed: First dangerous symptom not found, but later one is found.
All test cases passed!
Mark: 6 / 6
Test passed: One patient, one med.
Te

# Prepare Submission

To get full credit for this assignment, you should submit your assignment in two formats so that we can easily grade and debug your code:
1. **.ipynb:** First, confirm that your code can run from start to finish without any errors. To check this, go to "Runtime" > "Run all" in the Google Colab menu. If everything looks good, you can export your file by going to "File" > "Download" > "Download .ipynb".
2. **.pdf:** Run the function called `colab2pdf()` below. This will automatically convert your notebook to a PDF. Note that while "File" > "Print" > "Save as PDF" also works, it requires you to manually expand all of the cells and may cut off some images.

In [19]:
colab2pdf()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>